In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# 配置
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 超参数
num_epochs = 4
batch_size = 4
learning_rate = 0.001

In [6]:
# 数据集
import os
os.chdir('D:/User/Documents/ImportantData/LLMStudy/PyTorch/个人实践项目/251114 残差连接')
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_dataset = torchvision.datasets.CIFAR10(root='.', train=True,
                                             download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='.', train=False,
                                            download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                          shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                         shuffle=False)
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


nn.ConV2d()
输入四维张量[N, C, H, W]，N为batch大小，C为通道数，H为高，W为宽,
输出四维张量[N, out_channels, out_height, out_width]
实例化需要至少3个参数：输入通道数，输出通道数，卷积核大小

In [35]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# 残差网络基础块
class Conv2dModule(nn.Module):
    def __init__(self, in_channels, out_channels, id: int = 1, stride=1, pool=False):
        super(Conv2dModule, self).__init__()
        self.id = id
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.is_pool = pool
        self.pool = nn.MaxPool2d(2, 2)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        if self.is_pool:
            x = self.pool(x)
        x = self.conv2(x)
        x = self.bn2(x)
        #x = self.relu(x)
        return x


class ResNetModule(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, pool = False):
        super(ResNetModule, self).__init__()
        self.is_pool = pool
        self.pool = nn.MaxPool2d(2, 2)
        self.conv = Conv2dModule(in_channels, out_channels, stride=stride, pool = pool)
        self.relu = nn.ReLU()


    def forward(self, x):
        residual = x
        out = self.relu(x)
        out = self.conv(out)
        if self.is_pool:
            residual = self.pool(residual)
        out += residual
        return out


# 定义卷积网络
class ResConvNet2d(nn.Module):
    def __init__(self):
        super(ResConvNet2d, self).__init__()

        # 构建初始层
        self.first_layer = Conv2dModule(3, 64, pool = True) # 32*32 - 16*16

        # 构建中间隐藏层
        self.hidden_layer_1 = nn.ModuleList([
            ResNetModule(64, 64) for _ in range(3)
        ])

        self.hidden_layer_2 = nn.ModuleList([
            Conv2dModule(64,128,pool = True), # 16*16-8*8
            *[ResNetModule(128, 128) for _ in range(3)]
        ])

        # 全连接层
        self.adaptive_pool = nn.AdaptiveAvgPool2d((4, 4))  # 自适应池化确保统一尺寸
        self.classifier = nn.Sequential(
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(128 * 4 * 4, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 10)
        )

    def forward(self, x):
        x = self.first_layer(x)
        # 遍历 hidden_layer_1 中的所有模块
        for layer in self.hidden_layer_1:
            x = layer(x)
        # 遍历 hidden_layer_2 中的所有模块
        for layer in self.hidden_layer_2:
            x = layer(x)
        x = self.adaptive_pool(x)
        x = self.classifier(x)
        return x


In [36]:
# 创建模型
model = ResConvNet2d().to(device)

# 损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [37]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        # 前向传播
        outputs = model(images)
        # 计算损失
        loss = criterion(outputs, labels)
        # 后向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 打印结果
        if (i+1) % 2000 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

print('Finished Training')

Epoch [1/4], Step [2000/12500], Loss: 1.5233
Epoch [1/4], Step [4000/12500], Loss: 2.1281
Epoch [1/4], Step [6000/12500], Loss: 1.4091
Epoch [1/4], Step [8000/12500], Loss: 1.2140
Epoch [1/4], Step [10000/12500], Loss: 0.6425
Epoch [1/4], Step [12000/12500], Loss: 1.1706
Epoch [2/4], Step [2000/12500], Loss: 1.5883
Epoch [2/4], Step [4000/12500], Loss: 0.5850
Epoch [2/4], Step [6000/12500], Loss: 1.3667
Epoch [2/4], Step [8000/12500], Loss: 1.3554
Epoch [2/4], Step [10000/12500], Loss: 1.0557
Epoch [2/4], Step [12000/12500], Loss: 1.1747
Epoch [3/4], Step [2000/12500], Loss: 1.3481
Epoch [3/4], Step [4000/12500], Loss: 1.2817
Epoch [3/4], Step [6000/12500], Loss: 1.2034
Epoch [3/4], Step [8000/12500], Loss: 1.0309
Epoch [3/4], Step [10000/12500], Loss: 0.0989
Epoch [3/4], Step [12000/12500], Loss: 0.7476
Epoch [4/4], Step [2000/12500], Loss: 0.4701
Epoch [4/4], Step [4000/12500], Loss: 0.2958
Epoch [4/4], Step [6000/12500], Loss: 0.2662
Epoch [4/4], Step [8000/12500], Loss: 0.3796
Epoc

In [40]:
torch.save(model, './checkpoint.pth')

In [38]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    accuracy = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {accuracy} %')
    for i in range(10):
        accuracy = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {accuracy} %')


Accuracy of the network: 75.9 %
Accuracy of plane: 75.9 %
Accuracy of car: 80.9 %
Accuracy of bird: 72.1 %
Accuracy of cat: 53.7 %
Accuracy of deer: 66.1 %
Accuracy of dog: 69.7 %
Accuracy of frog: 89.3 %
Accuracy of horse: 77.4 %
Accuracy of ship: 87.1 %
Accuracy of truck: 86.8 %
